In [1]:
import ipyrad as ip

DEBUG:ipyrad:H4CKERZ-mode: __loglevel__ = DEBUG


In [17]:
data1 = ip.Assembly("Ron")


  New Assembly: Ron


In [18]:
data1.set_params("working_directory", "Ron")
data1.set_params("sorted_fastq_path", "/home/deren/Downloads/Ron/*.gz")
data1.set_params("datatype", "pairddrad")
data1.set_params("restriction_overhang", ("", ""))
data1.set_params("filter_adapters", 1)
data1.set_params("edit_cutsites", (6, 4))

data1.get_params()

  1   working_directory           ./Ron                                        
  2   raw_fastq_path              ./*.fastq                                    
  3   barcodes_path               ./*.barcodes.txt                             
  4   sorted_fastq_path           /home/deren/Downloads/Ron/*.gz               
  5   assembly_method             denovo                                       
  6   reference_sequence                                                       
  7   datatype                    pairddrad                                    
  8   restriction_overhang        ('', '')                                     
  9   max_low_qual_bases          5                                            
  10  phred_Qscore_offset         33                                           
  11  mindepth_statistical        6                                            
  12  mindepth_majrule            6                                            
  13  maxdepth                    1000  

In [21]:
data1.link_fastqs()
print data1.stats

                                   state  reads_raw
12_RE-05-11-4_CU_AATGGC-CGCGTCTA       1    3981908
34_RC-06-3-32_STX_AATGGC-TTGCACCC      1    4942688


In [ ]:
data1.step2(preview=True)
print data1.stats

INFO:ipyrad.core.assembly:try 10: starting controller
DEBUG:ipyrad.core.assembly:OK! Connected to (4) engines
DEBUG:ipyrad.assemble.util:zcat splittin' 34_RC-06-3-32_STX_AATGGC-TTGCACCC_R1_.fastq.gz
INFO:ipyrad.assemble.util:optim = 4942800
